In [1]:
#pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 376.0 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=8ae5077d343c68ef593ff48732a9cde2c0c8e0d3bc9f8e42568d9bcabd31be74
  Stored in directory: /home/jovyan/.cache/pip/wheels/aa/7d/7c/d47bc6347b27804958165fa526d1b50a0c76bae5545c866f99
Successfully built neo4j
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

path = '/home/jovyan/data/exoplanets.csv'

# Carregar o CSV com os tipos de dados especificados
df = pd.read_csv(path, comment='#', low_memory=False)

# Exibir as primeiras linhas do DataFrame para verificar a importação
print(df.head())

   rowid   pl_name hostname pl_letter    hd_name   hip_name         tic_id  \
0      1  11 Com b   11 Com         b  HD 107383  HIP 60202   TIC 72437047   
1      2  11 Com b   11 Com         b  HD 107383  HIP 60202   TIC 72437047   
2      3  11 Com b   11 Com         b  HD 107383  HIP 60202   TIC 72437047   
3      4  11 UMi b   11 UMi         b  HD 136726  HIP 74793  TIC 230061010   
4      5  11 UMi b   11 UMi         b  HD 136726  HIP 74793  TIC 230061010   

                        gaia_id  default_flag  sy_snum  ...  sy_kepmag  \
0  Gaia DR2 3946945413106333696             0        2  ...        NaN   
1  Gaia DR2 3946945413106333696             0        2  ...        NaN   
2  Gaia DR2 3946945413106333696             1        2  ...        NaN   
3  Gaia DR2 1696798367260229376             1        1  ...        NaN   
4  Gaia DR2 1696798367260229376             0        1  ...        NaN   

    rowupdate  pl_pubdate releasedate  pl_nnotes st_nphot st_nrvc st_nspec  \
0  2014-

In [3]:
# Criando o DataFrame de estrelas
df_stars = df[['hostname', 'st_teff', 'st_mass', 'st_rad']].drop_duplicates()
df_stars.columns = ['nome', 'st_teff', 'st_mass', 'st_rad']
df_stars.insert(0, 'estrelaId', range(1, len(df_stars) + 1))
df_stars.to_csv('/home/jovyan/data/estrelas.csv', index=False)

In [4]:
# Criando o DataFrame de planetas 
df_planets = df[['pl_name', 'pl_rade', 'pl_masse', 'pl_eqt', 'hostname']]
df_planets.columns = ['nome', 'raio', 'massa', 'eqtemp', 'estrelaNome']
df_planets = pd.merge(df_planets, df_stars[['estrelaId', 'nome']], left_on='estrelaNome', right_on='nome', how='left')
df_planets_final = df_planets[['nome_x', 'raio', 'massa', 'eqtemp', 'estrelaId']]
df_planets_final.columns = ['nome', 'raio', 'massa', 'eqtemp', 'estrelaId']
df_planets_final.to_csv('/home/jovyan/data/planetas.csv', index=False)

In [5]:
# Criando o DataFrame de observatórios
df_obs = df[['disc_facility']].drop_duplicates()
df_obs.columns = ['nome']
df_obs.insert(0, 'observatorioId', range(1, len(df_obs) + 1))
df_obs.to_csv('/home/jovyan/data/observatorios.csv', index=False)

In [7]:
# Criando o arquivo de relações entre planetas e estrelas (orbitas.csv)
df_orbitas = df_planets_final[['nome', 'estrelaId']]
df_orbitas.columns = [':START_ID(Planeta)', ':END_ID(Estrela)']
df_orbitas.insert(2, 'tipo', 'ORBITA')
df_orbitas.to_csv('/home/jovyan/data/orbitas.csv', index=False)

In [8]:
# Criando o arquivo de relações entre planetas e observatórios (descobertas.csv)
df_descobertas = pd.merge(df_planets_final[['nome', 'estrelaId']], df[['pl_name', 'disc_facility']], left_on='nome', right_on='pl_name', how='left')
df_descobertas_final = pd.merge(df_descobertas, df_obs[['observatorioId', 'nome']], left_on='disc_facility', right_on='nome', how='left')
df_descobertas_final = df_descobertas_final[['nome_x', 'observatorioId']]
df_descobertas_final.columns = [':START_ID(Planeta)', ':END_ID(Observatorio)']
df_descobertas_final.insert(2, 'tipo', 'DESCOBERTO_POR')
df_descobertas_final.to_csv('/home/jovyan/data/descobertas.csv', index=False)